## READ IN DATA

In [0]:
import re
import pandas as pd
import numpy as np


headers=['domain', 'path', 'size', 'ip', 'foo', 'bar', 'datetime', 'request', 'status', 'size2', 'referer', 'user_agent']
regex = r'(?:(.*?) (.*) (\d+?|-) )?(\d+\.\d+\.\d+\.\d+|-) (.*?) (.*?) \[(.*?)\] "(.*?[^\\])" (.*?) (\d+?|-)(?: "(.*?)" "(.*?)")?$'

array = []
l=0

with open("short.txt","r") as file:
    for line in file:        
        values = []
        if re.match(regex, line) is not None:
            m = re.match(regex, line)
            for i in range(1,13):  
                if m.group(i) is not None:
                    values.append(m.group(i)) 
                else:
                    values.append('None')
                    #print(values[i])
        
        else:
            print(l+1)
        
        array.append(values)        
        l+=1

## MAKE DATA FRAME

In [0]:
df = pd.DataFrame(array, columns=headers)
df.head()
#df.dtypes

## Cleaning

In [0]:
#CONVERT DATE TIME
from datetime import datetime

f = '%d/%b/%Y:%H:%M:%S %z'
time_conv = lambda x: datetime.strptime(x, f)
df['datetime']=df['datetime'].apply(time_conv)

#CONVERT TO NUMERIC
df[['size','size2','status']] = df[['size','size2','status']].apply(pd.to_numeric, errors="coerce")

#DIVIDE BY COLUMN DETAILS (BOTS OR NOT)
df['bot_or_not']= df['user_agent'].str.contains('bot')

#CHANGE TIME FROM EARLIEST TIME
df['delta_t']=df['datetime']-np.min(df['datetime'])

#REMOVE UNNECESSARY COULMNS
df=df.drop(columns='foo')
df=df.drop(columns='bar')
df=df.drop(columns='size2')
df.head()

# Split the request

In [0]:
regex_request = r'(?:(?:(GET|POST|PUT|HEAD|DELETE|PATCH|OPTIONS|TRACE))?(?: ?)(?:(.*))?(?: ))?(?:(.*))?'

request_method,request_content,request_version= [],[],[]

for item in df['request']:
    if re.match(regex_request, item) is not None:
        rq = re.match(regex_request, item)
        request_method.append(rq.group(1))
        request_content.append(rq.group(2))
        request_version.append(rq.group(3))
    else:
        request_method,request_content,request_version = None,None,None

df['request_method'],df['request_content'],df['request_version'] = request_method,request_content,request_version

df.head()

## SOME RESEARCH ON THE REQUEST CONTENT

### XML RPC

In [0]:
#XML RPC
#DIVIDE BY COLUMN DETAILS (WITH XML RPC OR NOT)
df['xmlrpc']= df['request_content'].str.contains('xmlrpc.php')
df.head()

In [0]:
#If there is Wordpress installed on the website
#It can be supspicious
df.index[df['xmlrpc'] == True].tolist()

In [0]:
### LOOK ALSO FOR 200 STATUS
#Hackers, probably
df.index[(df["xmlrpc"] == True) & (df["status"] == 200)].tolist()

In [0]:
#CREATE A COLUMN XMLRPC_WARNING
xmlrpc_warning = []
for index,value in enumerate(df['xmlrpc']):
    if df['xmlrpc'].iloc[index] == True and df["status"].iloc[index] == 200:
        xmlrpc_warning.append(True)
    else:
        xmlrpc_warning.append(False)

df['xmlrpc_warning'] = xmlrpc_warning

In [0]:
df['xmlrpc_warning']

In [0]:
#Hackers, probably ! 
#They try to determine if you’re running a piece of software, or if you are running a specific version of some software.
df.index[(df["xmlrpc"] == True) & (df["status"] == 404)].tolist()

In [0]:
#CREATE A COLUMN XMLRPC_SERIOUS
xmlrpc_serious = []
for index,value in enumerate(df['xmlrpc']):
    if df['xmlrpc'].iloc[index] == True and df["status"].iloc[index] == 404:
        xmlrpc_serious.append(True)
    else:
        xmlrpc_serious.append(False)

df['xmlrpc_serious'] = xmlrpc_serious

### SEARCHNIG FOR SOME INJECTION

In [0]:
regrex_login='pass|password|Password|Pass|pswd|pwd|username|Username'
regex_sudo = 'sudo'
regex_a2 = "/(\')|(\%27)|(\-\-)|(#)|(\%23)/ix"
regex_sql = '/(\')|(\%27)|(\-\-)|(#)|(\%23)/ix'
regex_sql2="/\w*((\%27)|(\'))((\%6F)|o|(\%4F))((\%72)|r|(\%52))/ix"
regex_xss = "/((\%3C)|<)((\%2F)|\/)*[a-z0-9\%]+((\%3E)|>)/ix" 
regex_os = "/(\||%00|system\(|eval\(|`|\\)/i"
regex_xss_img = "/((\%3C)|<)((\%69)|i|(\%49))((\%6D)|m|(\%4D))((\%67)|g|(\%47))[^\n]+((\%3E)|>)/I"
regex_cd = '\/\.\./'
regex_groups = 'etc\/groups'
regex_sql_to_shell = '/exec(\s|\+)+(s|x)p\w+/ix'
regex_remote_file = '/(https?|ftp|php|data):/i'


df['SQL']= df['request_content'].str.contains(regex_sql)
df['login']= df['request_content'].str.contains(regrex_login)
df['sudo']= df['request_content'].str.contains(regex_sudo)
df['cd']= df['request_content'].str.contains(regex_cd)
df['groups'] = df['request_content'].str.contains(regex_groups)

df['SQL_union']=df['request_content'].loc[df['SQL']==True].str.contains('union')
df['a2_injection']=df['request_content'].str.contains(regex_a2)

df['sql_shell']=df['request_content'].str.contains(regex_sql_to_shell)
df['remote_file']=df['request_content'].str.contains(regex_remote_file)


'''
TO LOOK FOR
same ip , different domain, xmlrpc
short amount of time '''

     
df.head()

In [0]:
pd.set_option('display.max_colwidth', -1)
#df['request_content'].loc[df['remote_file'] == True]
df.loc[df['regex_xss']== True]

### SCALING THE XML RPC

In [0]:
xmlrpc_scale = []
for i, item in enumerate(df['xmlrpc']):
    if df['xmlrpc'].iloc[i] is True:
        xmlrpc_scale.append(1)
    else:
        xmlrpc_scale.append(0)

#For the xml rpc warning
warning = df.index[df['xmlrpc_warning'] == True]
for item in warning:
    xmlrpc_scale[item] = 2
    #print(xmlrpc_scale[item])
    
#For the xml rpc warning
serious = df.index[df['xmlrpc_serious'] == True]
for item in serious:
    xmlrpc_scale[item] = 3
    #print(xmlrpc_scale[item])

df['xmlrpc_scale']= xmlrpc_scale

In [0]:
import collections
freq_req_xmlrpc = (collections.Counter(df['xmlrpc_scale']))
freq_req_xmlrpc

In [0]:
# Grouping per hour and type
df2 = df.groupby('datetime', df['xmlrpc_scale']).count().unstack()
# Droping unnecessary column level
df2.columns = df2.columns.droplevel()
df2.plot(kind='bar')

In [0]:
df.groupby('datetime')

In [0]:
df.loc[df['xmlrpc_scale'] == 3]